In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()g standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

## NOTE (1) 
Run the piece of code bellow to: <br>
-> produce all possible results of binary combinations

In [15]:
# Code imported form 
# https://stackoverflow.com/questions/14931769/how-to-get-all-combination-of-n-binary-value
import itertools
def binaryCombinations(N): 
    lst = list(itertools.product([0, 1], repeat=N))
    return lst

## NOTE (2)
### Code explanation about **storing measurements** : 

~~~ 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run( circuit_name , shots = N, memory = True )
    output = experiment.result().get_memory()[ i ] 
    
    # usually with N = 1, i = 0
~~~

<br>First, by
~~~ 
    simulation = Aer.get_backend('aer_simulator')
~~~
we connect our backend to the Aer simulator in Qiskit, in order to simulate quantum circuits. <br>
Then, with the command
~~~ 
    experiment = simulation.run( circuit_name , shots = N, memory = True )
~~~
we are executing N experiments running our circuit. It is important to set as True the last parameter, refering to memory. This creates an array of N positions, each one of them saving an experiment execution. <br>
Lastly, we use the command 
~~~ 
    output = experiment.result().get_memory()[ i ] 
~~~
to access measurement result of execution No. (i+1), which is stored in the i-th position of the (zero-based) array. 

In [127]:
# THE "AND_2_4()" WORKS ONLY FOR N = 2 OR N = 4

def AND_2_4(inputs): 
    # 0. GATE FUNCTIONALITY / LOGIC
    # If all inputs are '1', it returns '1'
    # If at least one input is '0', it returns '0'
    # _________________________________
    
    # 1. CREATE QUANTUM CIRCUIT
    # AND gate needs at least 2 qubits as input line -- more are acceptable. 
    # For N main inputs, we need an extra qubit to make measurements, 
    # so we will use N+1 lines of qubits. 
    # It also needs a classical bit to store measurement result.
    count, i = len(inputs), int(len(inputs)/2)
    while ( i > 0 ):
        count = count + i
        i = int(i/2)
    AND_gate = QuantumCircuit(count,1)
    
    # 2. INTIALIZE CIRCUIT TO MATCH FUNCTION INPUT
    # Due to Qiskit initializing qubits to |0>, 
    # we need to add an X gate to each input that is '1'
    for i in range(len(inputs)):
        if (inputs[i] == 1):
            AND_gate.x(i)
    AND_gate.barrier()

    # 3. ADD GATES TO ACHIEVE DESIRED OUTPUT
    entagl = [ [0, len(inputs)-1] , [len(inputs), count-2] ] 
    k = 0
    while ( k < len(entagl) ): 
        ind = 0
        for i in range(entagl[k][0], entagl[k][1], 2): 
            AND_gate.ccx(i,i+1,entagl[k][1]+1+ind)
            ind = ind + 1
        k = k + 1
    #AND_gate.ccx(count-3, count-2, count-1)
    AND_gate.measure(count-1, 0)
    AND_gate.draw()
    
    # 4. SIMULATE MEASUREMENT AND RETURN RESULTS 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run(AND_gate, shots=1, memory=True)
    output = experiment.result().get_memory()[0]
    
    return AND_gate, int(output)

In [128]:
# Executing all possible results for AND gate
# using N qubits as input
# THE "AND_2_4()" WORKS ONLY FOR N = 2 OR N = 4
N = 2
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = AND_2_4(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER AND gate: "',output_value, '"')
    display(output_qc.draw('text'))
    print('\n')

BEFORE:  " (0, 0) " -->  AFTER AND gate: " 0 "


░         
q_0: ─░───■─────
      ░   │     
q_1: ─░───■─────
      ░ ┌─┴─┐┌─┐
q_2: ─░─┤ X ├┤M├
      ░ └───┘└╥┘
c: 1/═════════╩═
              0



BEFORE:  " (0, 1) " -->  AFTER AND gate: " 0 "


░         
q_0: ──────░───■─────
     ┌───┐ ░   │     
q_1: ┤ X ├─░───■─────
     └───┘ ░ ┌─┴─┐┌─┐
q_2: ──────░─┤ X ├┤M├
           ░ └───┘└╥┘
c: 1/══════════════╩═
                   0



BEFORE:  " (1, 0) " -->  AFTER AND gate: " 0 "


┌───┐ ░         
q_0: ┤ X ├─░───■─────
     └───┘ ░   │     
q_1: ──────░───■─────
           ░ ┌─┴─┐┌─┐
q_2: ──────░─┤ X ├┤M├
           ░ └───┘└╥┘
c: 1/══════════════╩═
                   0



BEFORE:  " (1, 1) " -->  AFTER AND gate: " 1 "


┌───┐ ░         
q_0: ┤ X ├─░───■─────
     ├───┤ ░   │     
q_1: ┤ X ├─░───■─────
     └───┘ ░ ┌─┴─┐┌─┐
q_2: ──────░─┤ X ├┤M├
           ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

In [129]:
# Executing all possible results for AND gate
# using N qubits as input
# THE "AND_2_4()" WORKS ONLY FOR N = 2 OR N = 4
N = 4
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = AND_2_4(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER AND gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

BEFORE:  " (0, 0, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 1, 1) " -->  AFTER AND gate: " 1 "


In [130]:
# THE "AND_3()" WORKS ONLY FOR N = 3

def AND_3(inputs): 
    # 0. GATE FUNCTIONALITY / LOGIC
    # If all inputs are '1', it returns '1'
    # If at least one input is '0', it returns '0'
    # _________________________________
    
    # 1. CREATE QUANTUM CIRCUIT
    # AND gate needs at least 2 qubits as input line -- more are acceptable. 
    # For N main inputs, we need an extra qubit to make measurements, 
    # so we will use N+1 lines of qubits. 
    # It also needs a classical bit to store measurement result.
    count, i = len(inputs), int(len(inputs)/2)
    while ( i > 0 ):
        count = count + i
        i = int(i/2)
    if (len(inputs)%2 == 1):
        count = count + 1
    AND_gate = QuantumCircuit(count,1)
    
    # 2. INTIALIZE CIRCUIT TO MATCH FUNCTION INPUT
    # Due to Qiskit initializing qubits to |0>, 
    # we need to add an X gate to each input that is '1'
    for i in range(len(inputs)):
        if (inputs[i] == 1):
            AND_gate.x(i)
    AND_gate.barrier()

    # 3. ADD GATES TO ACHIEVE DESIRED OUTPUT
    entagl = [ [0, len(inputs)-1] , [len(inputs), count-2] ] 
    k = 0
    while ( k < len(entagl) ): 
        ind = 0
        for i in range(entagl[k][0], entagl[k][1], 2): 
            AND_gate.ccx(i,i+1,entagl[k][1]+1+ind)
            ind = ind + 1
        k = k + 1
    AND_gate.ccx(count-3, count-2, count-1)
    AND_gate.measure(count-1, 0)
    AND_gate.draw()
    
    # 4. SIMULATE MEASUREMENT AND RETURN RESULTS 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run(AND_gate, shots=1, memory=True)
    output = experiment.result().get_memory()[0]
    
    return AND_gate, int(output)

In [131]:
# Executing all possible results for AND gate
# using N qubits as input
# THE "AND_3()" WORKS ONLY FOR N = 3
N = 3
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = AND_3(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER AND gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

BEFORE:  " (0, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (1, 1, 1) " -->  AFTER AND gate: " 1 "


In [132]:
def AND(inputs): 
    inpNo = len(inputs)
    if (inpNo == 2 or inpNo == 4):
        return AND_2_4(inputs)
    elif (inpNo == 3): 
        return AND_3(inputs)
    else: 
        AND_1, output1 = AND(inputs[0:int(inpNo/2)])
        AND_2, output2 = AND(inputs[int(inpNo/2):inpNo])
        AND_gate, output = AND([int(output1), int(output2)])
    
    return AND_gate, int(output)

In [133]:
# Executing all possible results for AND gate
# using N qubits as input
# WORKS FOR N = 2, 3, 4, 5, 6, 7, ...

N = 7
combinations = binaryCombinations(N)
no = len(combinations)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = AND(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER AND gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

BEFORE:  " (0, 0, 0, 0, 0, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 0, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 1) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1, 1, 0) " -->  AFTER AND gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1, 1, 1) " -->  AFTER AND gate: " 0 "
BEFORE: 

In [ ]:
# Executing all possible results for AND gate
# using N qubits as input
# WORKS FOR N = 2, 3, 4, 5, 6, 7, 8 ...

N = 8
combinations = binaryCombinations(N)
no = len(combinations)

print ("For N = ", N, ", show ONLY input combinations that return 1 as output")
for comb in combinations: 
    init_value = comb
    output_qc, output_value = AND(init_value)
    if ( output_value == 1): 
        print('BEFORE:  "',init_value,'" -->  AFTER AND gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

For N =  8 , show ONLY input combinations that return 1 as output
